# Usage of trained models for prediction of potential additive properties
Import relevant libraries

In [ ]:
from bayesian_optimization.MyGraphConvModel import MyGraphConvModel # customized GraphConvModel
import deepchem as dc

Restore models from curently saved folders

In [ ]:
SEI_model = MyGraphConvModel(n_tasks=4, graph_conv_layers=[465, 465], dense_layers=[342, 342, 342], dropout=0.016595, \
learning_rate=0.0018738, uncertainty=True, model_dir="final_SEI_model")
SEI_model.restore()

LL_model = MyGraphConvModel(n_tasks=4, graph_conv_layers=[545, 545], dense_layers=[562, 562, 562, 562], dropout=0.030050724302585315, \
learning_rate=0.0006959499230450821, uncertainty=True, model_dir="final_LL_model")
LL_model.restore()

Use the models to make predictions

In [ ]:
smiles = ["FC(F)(F)C(=O)CC(=O)C(F)(F)F", "CS(=O)(=O)C"]
featurizer=dc.feat.ConvMolFeaturizer()
loader = dc.data.InMemoryLoader(tasks=[None], featurizer=featurizer)
dataset = loader.create_dataset(smiles)
predictions, uncertainty = SEI_model.predict_uncertainty(dataset)
print(f"SEI model:\nThe predicted LUMO energy (eV) are: {predictions[:,0]}\n"
      f"The predicted (100) interaction energy (eV) are {predictions[:,1]}\n"
      f"The predicted (110) interaction energy (eV) are {predictions[:,2]}\n"
      f"The predicted (111) interaction energy (eV) are {predictions[:,3]}\n")
predictions, uncertainty = LL_model.predict_uncertainty(dataset)
print(f"LL model:\nThe predicted LUMO energy (eV) are: {predictions[:,0]}\n"
      f"The predicted (100) interaction energy (eV) are {predictions[:,1]}\n"
      f"The predicted (110) interaction energy (eV) are {predictions[:,2]}\n"
      f"The predicted (111) interaction energy (eV) are {predictions[:,3]}\n")

Visualize the molecule by drawing its structure

In [ ]:
from IPython.display import SVG, display
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw.MolDrawing import DrawingOptions

for s in smiles:
    mol = Chem.MolFromSmiles(s)
    rdDepictor.Compute2DCoords(mol)  # for generating conformer ID
    # create a drawer container
    drawer = rdMolDraw2D.MolDraw2DSVG(300, 300)
    # define drawer options
    drawer.drawOptions().updateAtomPalette({k: (0, 0, 0) for k in DrawingOptions.elemDict.keys()})
    drawer.SetLineWidth(2)
    drawer.SetFontSize(1.0)
    drawer.DrawMolecule(mol)
    drawer.FinishDrawing()
    # generate and write the svg strings
    svg = drawer.GetDrawingText().replace('svg:', '')
    display(SVG(svg))